# MCP Historical Weather Comparison

## Overview

This notebook demonstrates how to use **Model Context Protocol (MCP)** to extend Claude's capabilities with real-time data access. We'll build a system that allows Claude to access historical weather data and compare annual weather statistics between two locations.

### What is MCP?

Model Context Protocol (MCP) is a standard that enables AI models to securely connect with external data sources and tools. Instead of being limited to training data, models can access live information, APIs, and services.

### What You'll Learn

1. **Limitations of LLMs without external tools** - See how Claude responds to weather queries without access to real data
2. **MCP Integration** - Connect Claude to a historical weather API
3. **Data Visualization** - Create aesthetically pleasing charts comparing weather patterns
4. **Interactive Analysis** - Ask Claude to analyze and compare weather data between locations

### Goals

By the end of this notebook, you'll have a working system that can:
- Fetch historical weather data for any location
- Compare annual weather statistics between two cities
- Generate visualizations and insights about weather patterns
- Demonstrate the power of extending LLMs with external data sources

## Setup

### Environment Setup

This notebook is designed to work in both Google Colab and local Jupyter environments. The following cell will automatically detect the environment and install the necessary dependencies.

In [1]:
import sys

# Check if we're running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

# Install required packages
if IN_COLAB:
    # Install packages for Colab environment
    packages = [
        "anthropic",
        "altair",
        "pydantic",
        "requests"
    ]
    !pip install {" ".join(packages)}

### Python Package Imports

Now let's import all the necessary packages for our weather comparison system.

In [2]:
import os
import json
import getpass
from typing import Optional, Dict, Any
from datetime import datetime, timedelta

# Data handling and validation
import requests
from pydantic import BaseModel, Field, validator

# Visualization
import altair as alt
alt.data_transformers.enable('json')

# Claude API
from anthropic import Anthropic

print("All packages imported successfully!")

All packages imported successfully!


### Anthropic API Key Setup

We need to securely obtain your Anthropic API key to interact with Claude. This function will try multiple methods in order of preference for security.

In [3]:
def get_api_key():
    try:
        if IN_COLAB:
            # Import package for accessing user data
            from google.colab import userdata
            api_key = userdata.get('ANTHROPIC_API_KEY')
        else:
            api_key = os.environ.get("ANTHROPIC_API_KEY")
    except:
        # Prompt user for their API key
        api_key = getpass.getpass("Enter your Anthropic API key: ")
    return api_key

## Motivation: Do we actually need to extend LLMs?

Before diving into MCP integration, let's first see what happens when we ask Claude to compare weather data between two locations **without giving it access to any external tools or data sources**.

This will demonstrate the fundamental limitation of LLMs: they can only work with information from their training data, which has a knowledge cutoff and may not include specific, current, or detailed data.

In [4]:
def ask_claude_without_tools(question: str) -> str:
    """
    Send a question to Claude without any external tools or data access.
    
    Args:
        question: The question to ask Claude
    
    Returns:
        Claude's response as a string
    """
    try:
        client = Anthropic(api_key=get_api_key())
        message = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1000,
            messages=[
                {
                    "role": "user", 
                    "content": question
                }
            ]
        )
        return message.content[0].text
    except Exception as e:
        return f"Error: {e}"

# Test Claude's response without external data
weather_question = """
Please compare the annual weather statistics for San Francisco, California and Miami, Florida for the year 2023. 
I'd like to see:

1. Average temperatures by year
2. Total precipitation by year  
3. Number of sunny days by year
4. Humidity levels by year
5. Any notable weather patterns or extremes

Please provide specific data and create a comparison showing which city had more favorable weather conditions.
"""

print("🤖 Asking Claude about weather data WITHOUT external tools...")
print("="*70)
print(f"Question: {weather_question}")
print("="*70)
print("Claude's Response:")
print()

response = ask_claude_without_tools(weather_question)
print(response)

🤖 Asking Claude about weather data WITHOUT external tools...
Question: 
Please compare the annual weather statistics for San Francisco, California and Miami, Florida for the year 2023. 
I'd like to see:

1. Average temperatures by year
2. Total precipitation by year  
3. Number of sunny days by year
4. Humidity levels by year
5. Any notable weather patterns or extremes

Please provide specific data and create a comparison showing which city had more favorable weather conditions.

Claude's Response:

I don't have access to specific 2023 weather data for San Francisco and Miami. However, I can provide you with typical annual weather patterns for these cities based on historical averages, which will give you a good comparison framework:

## Typical Annual Weather Comparison

### 1. Average Temperatures
**San Francisco:**
- Annual average: 57-59°F (14-15°C)
- Summer highs: 65-70°F (18-21°C)
- Winter lows: 45-50°F (7-10°C)

**Miami:**
- Annual average: 76-78°F (24-26°C)
- Summer highs: 85-9

### Key Observations

As you can see from Claude's response above, without access to external data sources, the model has several limitations:

1. **No real-time data**: Claude can't access current or specific historical weather data
2. **General knowledge only**: Responses are based on general patterns from training data
3. **No specific metrics**: Can't provide exact precipitation amounts, temperatures, or day counts
4. **No visualizations**: Can't create charts or graphs from actual data

This demonstrates why **Model Context Protocol (MCP)** is valuable - it bridges the gap between the model's reasoning capabilities and real-world data access.

## Analysis

*This section will contain analysis of weather data patterns and comparisons between locations once we implement the MCP integration.*

```mermaid
graph TD
    A[Weather API] --> B[MCP Server]
    B --> C[Claude Client]
    C --> D[Data Analysis]
    D --> E[Visualization]
    E --> F[User Insights]
```

**Coming soon:**
- Historical weather data retrieval
- Statistical analysis of temperature and precipitation patterns
- Seasonal trend identification
- Comparative analysis between locations